In [1]:
import torch
import os
from transformers import SwinForImageClassification, AutoImageProcessor

/Users/jyo/Desktop/Projects/MedAgent/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Configuration ---
# 1. Define paths and names based on your training process
BASE_MODEL_ID = "microsoft/swin-large-patch4-window12-384-in22k"
PATH_TO_PTH_FILE = "swin_skin_disease_model.pth"
LOCAL_SAVE_PATH = "swin_model"

In [3]:
# 2. Define class labels (must match the labels used during training)
CLASS_LABELS = [
    '1. Enfeksiyonel', 
    '2. Ekzama', 
    '3. Akne', 
    '4. Pigment', 
    '5. Benign', 
    '6. Malign'
]
NUM_LABELS = len(CLASS_LABELS)

# Create the label-to-ID mappings for the config.json
id2label = {i: label for i, label in enumerate(CLASS_LABELS)}
label2id = {label: i for i, label in enumerate(CLASS_LABELS)}

# --- Device Setup (Optimized for M1) ---
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using Apple Metal Performance Shaders (MPS) device.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Using NVIDIA CUDA device.")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU device. Inference/Packaging will be slower.")

✅ Using Apple Metal Performance Shaders (MPS) device.


In [4]:
# --- Step 1 & 2: Load Base Model/Processor and Apply Weights ---
try:
    print(f"Loading base model structure: {BASE_MODEL_ID}")
    model = SwinForImageClassification.from_pretrained(
        BASE_MODEL_ID,
        num_labels=NUM_LABELS,
        ignore_mismatched_sizes=True  # Necessary because we are changing the head
    ).to(device)

    print(f"Loading weights from {PATH_TO_PTH_FILE}...")
    model.load_state_dict(torch.load(PATH_TO_PTH_FILE, map_location=device))
    model.eval() # Set model to evaluation mode

    print("Loading base image processor...")
    image_processor = AutoImageProcessor.from_pretrained(BASE_MODEL_ID)

except FileNotFoundError:
    print(f"\nERROR: Weights file '{PATH_TO_PTH_FILE}' not found.")
    print("Please place the .pth file in the same directory as this script.")
    exit()
except Exception as e:
    print(f"\nERROR during model/processor loading: {e}")
    exit()

Loading base model structure: microsoft/swin-large-patch4-window12-384-in22k


Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-large-patch4-window12-384-in22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1536]) in the checkpoint and torch.Size([6, 1536]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading weights from swin_skin_disease_model.pth...
Loading base image processor...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
# --- Step 3: Update Model Configuration ---
print("Adding label mappings to model configuration...")
model.config.id2label = id2label
model.config.label2id = label2id

Adding label mappings to model configuration...


In [6]:
# --- Step 4: Save All Necessary Files Locally ---
print(f"Saving complete model package to: {LOCAL_SAVE_PATH}")

# Create the directory if it doesn't exist
os.makedirs(LOCAL_SAVE_PATH, exist_ok=True)

# Save the model (this generates config.json and pytorch_model.bin)
model.save_pretrained(LOCAL_SAVE_PATH)

# Save the processor (this generates preprocessor_config.json)
image_processor.save_pretrained(LOCAL_SAVE_PATH)

print("\n--- Success! ---")
print("All necessary files have been created in the following directory:")
print(f"  {os.path.abspath(LOCAL_SAVE_PATH)}")

Saving complete model package to: swin_model

--- Success! ---
All necessary files have been created in the following directory:
  /Users/jyo/Desktop/Projects/MedAgent/swin_model


In [7]:
# --- Step 5: Verification (Optional but recommended) ---
print("\n--- Verification Step: Testing Local Load ---")
try:
    # Load from the newly created local folder
    reloaded_processor = AutoImageProcessor.from_pretrained(LOCAL_SAVE_PATH)
    reloaded_model = SwinForImageClassification.from_pretrained(LOCAL_SAVE_PATH).to(device)
    
    # Create a dummy input tensor (since we don't have a real image here)
    dummy_input = torch.randn(1, 3, 384, 384).to(device)
    
    with torch.no_grad():
        outputs = reloaded_model(dummy_input).logits
        
    predicted_idx = torch.argmax(outputs, dim=1).item()
    predicted_label = reloaded_model.config.id2label[predicted_idx]

    print("Local model loading and inference test successful.")
    print(f"Test Prediction: {predicted_label} (Index: {predicted_idx})")

except Exception as e:
    print(f"Verification FAILED. Could not load or test the saved model: {e}")


--- Verification Step: Testing Local Load ---
Local model loading and inference test successful.
Test Prediction: 1. Enfeksiyonel (Index: 0)
